# Taller de procesamiento de BigData en Spark + R
Manuel Parra (manuelparra@decsai.ugr.es). <a href="http://sci2s.ugr.es/es">Soft Computing and Intelligent Information Systems</a>
. <a href="http://sci2s.ugr.es/dicits/">Distributed Computational Intelligence and Time Series</a>. **University of Granada**.
![logos](https://sites.google.com/site/manuparra/home/header.png)

## Primer ejemplo  y toma de contacto con SparkR


Como siempre para todos nuestros `scripts` con **SparkR**, cargamos la biblioteca, y creamos una nueva sesión de SparkR.

In [ ]:
#Fijamos la ruta donde está instalado Spark
Sys.setenv("SPARK_HOME"='/usr/local/spark/')

.libPaths(c(file.path(Sys.getenv("SPARK_HOME"),"R/lib/"),.libPaths()))
# Añadimos la bibioteca
library(SparkR)
# Abrimos la conexión
sparkR.session(appName="Primeros_Pasos", master = "local[*]", sparkConfig = list(spark.driver.memory = "1g"))

Con **SparkR** podemos crear un `DataFrame` de Spark desde un `data.frame` habitual usado en R. 

Un ``DataFrame`` es una colección distribuida de datos organizada en columnas. 

Los ``dataframes`` son conceptualmente equivalentes a bases de datos relacionales o a `data.frames` en R o Python, pero con una ventaja: son mucho más eficientes para el trabajo con grandes volumenes de datos. Los ``DataFrames``pueden ser creados desde una amplio surtido de fuentes muy diferentes. Es decir, casi cualquier cosa puede ser un ``dataframe``, por ejemplo ficheros estructurados, tablas en ``HIVE``, bases de datos externas o RDDs.


Los RDDs son la principal abstracción de datos en Spark. Un RDD es una colección resilente y distribuida de registros. Esta es una de las claves de Spark y es uno de los componentes fundamentales del `core` de Spark.

In [ ]:
# Vamos a usar un dataset sencillo integrado en R
# El dataset contiene el tiempo de espera entre erupciones y duración 
# de la erupción de un geiser de Yellowstone
class(faithful)

# Convertimos un dataframe de R en un DataFrame de Spark, que llamaremos SparkDataFrame
df_faithful <- createDataFrame(faithful)

# Vemos el tipo de dataset nuevo
class(df_faithful)

# Visualizamos de forma rápida el contenido
head(df_faithful)

# Usamos la función printSchema de SparkR para 'deducir' el esquema de datos (la estructura)
printSchema(df_faithful)

Un ``SparkDataFrame`` puede ser registrado como una vista temporal en ``SparkSQL`` y que permite ejecutar sentencias SQL sobre los datos. La funcionalidad de SQL permite a las aplicaciones y flujos de trabajo ejecutar consultas SQL de forma programatica, devolviendo el resultado también como SparkDataFrame.

Esto es importante, ya que todas las transformaciones a los conjuntos de datos que están en formato SparkDataFrames, siguen siendo SparkDataFrames, lo que hace que toda su manipulación corra por parte de Spark con todas las ventajas que eso tiene:

- Volumen masivo de datos
- Almacenamiento distribuido
- Resilencia


## Spark es perezo

In [ ]:
df_iris <- createDataFrame(iris)

Ejecutamos los siguientes trozos de código en R:

In [ ]:
p <- proc.time()
df_filtrado <- filter(df_iris,df_iris$Species=="setosa")
proc.time()-p

In [ ]:
p <- proc.time()
count(df_filtrado)
proc.time()-p

Al ejecutar una función de tipo acción (en esta caso ``count``, ``print``, ``head``, etc.) lanza todo los procesos necesarios para conseguir realizar la acción pedida. En este caso sería hacer el filtro y luego contar. Esto se puede ver en Spark UI (la veremos más adelante).

In [ ]:
p <- proc.time()
count(df_filtrado)
proc.time()-p

Hace exactamente lo mismo, primer filtra y luego cuenta, ya que por defecto no hace persistente el DataFrame intermedio aunque lo estemos definiendo como ``df_filtrado``.

Si queremos hacer persistente un DataFrame intermedio podemos con la función de cacheado.


## Operaciones sencillas con SparkR sobre SparkDataFrames

En estos ejemplos vamos a tratar de ver una parte muy muy simple sobre la manipulación de los datos en el formato que entiende SparkR.

In [ ]:
# Contamos los elementos a partir de un filtro normal
count(filter(df_faithful,"eruptions>3.0"))

# Convertimos a vista temporal de datos en SparkSQL y le damos el nombre faithful a la 'tabla'
createOrReplaceTempView(df_faithful,"faithful")

# Usamos SparkSQL para hacer consultas a los datos.
eruptions_sql <- sql("SELECT eruptions FROM faithful WHERE eruptions >= 3.0")

# Contamos el resultado
count(eruptions_sql)

# Mostramos un resumen
head(eruptions_sql)

## Operaciones sobre los conjuntos de datos

La guía de referencia de funciones de la API de Spark con R se puede ver en: 

    https://spark.apache.org/docs/2.0.0-preview/api/R/

Veamos las más importantes y sus diferencias con las de R equivalentes.

Recordamos que siempre para trabajar con SparkR, tenemos que terminar la sesión de Spark.

## Interfaces de gestión de trabajos de Spark

Spark pone a disposición del administrador un sistema web que permite controlar el estado del cluster en cuando a jobs, workers, operaciones, ...


Gestión de nodos Workers:
- En http://192.168.99.10:8080 

Gestión de Jobs:
- En http://192.168.99.10:4040/



In [ ]:
sparkR.session.stop()

Con esta sentencia se cierra el contexto abierto en SparkR y se liberan todos los recursos.